# Sentiment analysis | Depp vs Heard

## Data extraction

Twitter extraction by keywords

### Imports

In [57]:
import tweepy as tw
import pandas as pd
import numpy as np
import re

### Twitter authentificathion

In [30]:
ACCESS_KEY = "1528407397403959296-SiXEMki1ykYG1qg3WPht56Dh1nxvfm"
ACCESS_SECRET = "6BBykwX8DUyDKYpQ46pT7RfPMBM3E7muynpOpYSjgqPwo"
CONSUMER_KEY= 'fVDe8jYU1oWQKkOZlVHd17DDR'
CONSUMER_SECRET= '3EPEAxlpGl48QYl2iNBk6HlbrjONLWQ5qoDY61uVReLeHgbX6V'
BEARER = "AAAAAAAAAAAAAAAAAAAAADTKdAEAAAAAEygerXyhYaLCb0wzIuc7wsOak7w%3DJ6QKl3DzawKIUGdQaArG8yovhy8LpUr0MTgovJQ01naJ5b2BRZ"

In [ ]:
client = tw.Client(bearer_token=BEARER)

### Tweets extraction

In [53]:
# Get tweets that contain the hashtag #petday
# -is:retweet means I don't want retweets
# lang:en is asking for the tweets to be in english
query = '#depp -is:retweet lang:en'
tweets = client.search_recent_tweets(query=query, tweet_fields=['created_at'], max_results=100)

In [54]:
depp_tweets = pd.DataFrame(tweets.data)
depp_tweets.drop_duplicates(inplace=True)
depp_tweets.dropna(inplace=True)
depp_tweets

,created_at,id,text
0,2022-05-31 16:17:20+00:00,1531671178309513217,Did you not listen to all the audio recordings...
1,2022-05-31 16:07:33+00:00,1531668717041483783,How will they decide? #Depp #DeppVHeardTrial #...
2,2022-05-31 15:50:45+00:00,1531664486372974593,Remove Amber Heard as ACLU Ambassador of Women...
3,2022-05-31 15:49:35+00:00,1531664196076769281,Jurors to return Tuesday to continue deliberat...
4,2022-05-31 15:33:56+00:00,1531660256039813124,@GellertDepp My take on this is that the quest...
...,...,...,...
95,2022-05-31 05:30:22+00:00,1531508364227796992,#JohnnyDeppIsInnocent #Depp the best revenge i...
96,2022-05-31 05:27:26+00:00,1531507623735992320,"#AmberHeard was sleeping with James Franco, he..."
97,2022-05-31 05:23:09+00:00,1531506546831654912,(The Independent):#Johnny #Depp joins Jeff Bec...
98,2022-05-31 05:10:56+00:00,1531503474533449728,Rum Diary- I have to admit.. I have never watc...


### Data cleaning

Remove tags "@xxx"

In [61]:
def remove_pattern(text, pattern_regex):
    r = re.findall(pattern_regex, text)
    for i in r:
        text = re.sub(i, '', text)

    return text

depp_tweets['text'] = np.vectorize(remove_pattern)(depp_tweets['text'], "@[\w]*: | *RT*")
depp_tweets

TypeError: expected string or bytes-like object

### Model

In [55]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

def fetch_sentiment_using_SIA(text):
    sid = SentimentIntensityAnalyzer()
    polarity_scores = sid.polarity_scores(text)
    if polarity_scores['neg'] > polarity_scores['pos']:
        return 'negative'
    else:
        return 'positive'

def fetch_sentiment_using_textblob(text):
    analysis = TextBlob(text)
    # set sentiment 
    if analysis.sentiment.polarity >= 0:
        return 'positive'
    else: 
        return 'negative'